In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np

df = pd.read_csv('data/train.csv')
pixels_col = [col for col in df.columns if col != 'label']
samples = df[pixels_col].values
df.shape

(42000, 785)

<p style="font-size:15px">The output is an integer from 0-9. we use one hot encoding transformaton on a number to make a binary matrix of size 1*10 becauce the output value of the neural netwrok would be a probablity number between 0 -1 for each of the posisble digits. <br />
example : number 3 --one_hot-encode-> [0,0,0,1,0,0,0,0,0,0,0] <br />
the output probablity would be like: [0,0.07, 0.2,0.7,0.02,0,0,0,0,0.01 ]</p>

In [6]:
def one_hot_digits(x):
    res = np.zeros((10,), dtype=float)
    res[x] = 1.0
    return(res)
target = np.apply_along_axis(one_hot_digits, axis=1, arr=df[['label']])
target.shape

(42000, 10)

<p style="font-size:15px">Spliting the training data to test and train sets: </p>

In [7]:
from sklearn.model_selection import train_test_split
train_samples, X_test, train_target, y_test = train_test_split(samples, target, test_size=0.2, random_state=4242)
train_target.shape

(33600, 10)

<h3>Deep Neural Network Model:</h3>

<p style="font-size:15px;">
The model is a deep neural network with three hidden layers with the 500 nodes each. <br />
layers weigh shape :  (N_samples X N_features)(input of the layer) * (weights of the layer)(N_features X N_outputs) => (outputof the layer)((N_samples X N_outputs) + biases(N_outputs X 1)) <br />
biases shape: N_outputs X 1 </p>

In [8]:
hl1_no_nodes = 500
hl2_no_nodes = 500
hl3_no_nodes = 500
n_classes = 10

hidden_layer_one = {'weights': tf.Variable(tf.random_normal(shape = [784, hl1_no_nodes])),
                    'biases': tf.Variable(tf.random_normal( shape = [hl1_no_nodes]))}
    
hidden_layer_two = {'weights': tf.Variable(tf.random_normal(shape = (hl1_no_nodes, hl2_no_nodes))),
                    'biases': tf.Variable(tf.random_normal( shape = [hl2_no_nodes]))}
    
hidden_layer_three = {'weights': tf.Variable(tf.random_normal(shape = [hl2_no_nodes, hl3_no_nodes])),
                      'biases': tf.Variable(tf.random_normal( shape = [hl3_no_nodes]))}
    
output_layer = {'weights': tf.Variable(tf.random_normal(shape = [hl3_no_nodes, n_classes])),
                'biases': tf.Variable(tf.random_normal( shape = [n_classes]))}
    


<p style="font-size:15px">For speeding up the training, we train images in batches in every traning cycle (epoch), here we pick the size of each batch 100. <br/> 
x, y are tensorflow variables used for input and target</p>

In [9]:
batch_size = 100
X = tf.placeholder(tf.float32, shape=(None, 784))
y = tf.placeholder(tf.float32)

<p style="font-size:15px">we apply a rectifier (rectifier linear unit:relu) which is an activation function to make the neurons in the hidden layers. <br />
(output of the layer) = ((N_samples X N_outputs) + biases(N_outputs X 1))
</p>

In [10]:
layer1_output = tf.add(tf.matmul(X, hidden_layer_one['weights']) , hidden_layer_one['biases'])
layer1_output = tf.nn.relu(layer1_output)

layer2_output = tf.add(tf.matmul(layer1_output , hidden_layer_two['weights']) , hidden_layer_two['biases'])
layer2_output = tf.nn.relu(layer2_output)

layer3_output = tf.add(tf.matmul(layer2_output, hidden_layer_three['weights']) , hidden_layer_three['biases'])
layer3_output = tf.nn.relu(layer3_output)

output = tf.matmul(layer3_output, output_layer['weights']) + output_layer['biases']

<h3>Training the neural network:</h3>

<p style="font-size:15px">softmax is a logistic function () which predcits the probability of the classes (similar to sigmoid function)
cross entropy: calcualtes the error or the difference between the predition and the target. <br />
And reduce_mean (without specifiying the axis of cal) it returns the mean of a tensor elements and  a single value <br />
and adamoptimizer minimizes the cost function</p>

In [18]:
prediction = output
#adding regulization:
beta = 0.01
regularizers =( tf.nn.l2_loss( hidden_layer_one['weights']) +  tf.nn.l2_loss(hidden_layer_two['weights']) + 
 tf.nn.l2_loss( hidden_layer_three['weights'] + tf.nn.l2_loss( output_layer['weights'])) )            
               
cost = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y)) 
        + beta * regularizers)

optimizer = tf.train.AdamOptimizer().minimize(cost)

<p style="font-size:15px">we extecute opration objects by running session: <br />
prediction or output is tensor of probablity, for finding the accryacy 
we can find the index of max value of probablity for each sample by using argmax and which gives
us the digit and compare it to the target value of the sample: <br />
we use 100 cycle of feedforward to train the samples: epoch=100
</p>

In [19]:
from sklearn.utils import shuffle

epochs = 100
is_correct = tf.equal(tf.argmax(prediction,1) , tf.argmax(y,1))    
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
#accuracy_test_b0 = []
#accuracy_train_b0 = []
# accuracy_test_b0_01 = []
# accuracy_train_b0_01 = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(1, epochs+1):
        epoch_loss = 0
        X_train, y_train = shuffle(train_samples, train_target, random_state=45)
        for _ in range(train_samples.shape[0]/batch_size):
           
            train_feed_dict = {X: X_train[(_*batch_size):((_+1)*batch_size)],
                         y:y_train[(_*batch_size):((_+1)*batch_size)]}
            _, c = sess.run([optimizer, cost], train_feed_dict)
            epoch_loss += c
        test_feed_dict = {X: X_test, y:y_test}
        accuracy_test = 100*sess.run(accuracy,feed_dict=test_feed_dict)
        #accuracy_train = sess.run(accuracy*100,feed_dict=train_feed_dict)
        #accuracy_test_b0.append(accuracy_test)
        #accuracy_train_b0.append(accuracy_train)
        print('Epoch', epoch, 'completed out of', epochs, 'loss:', epoch_loss, 'Accuracy:', accuracy_test)

('Epoch', 1, 'completed out of', 100, 'loss:', 1689442509824.0, 'Accuracy:', 85.523808002471924)
('Epoch', 2, 'completed out of', 100, 'loss:', 725722997888.0, 'Accuracy:', 88.047617673873901)
('Epoch', 3, 'completed out of', 100, 'loss:', 366764497152.0, 'Accuracy:', 89.166665077209473)
('Epoch', 4, 'completed out of', 100, 'loss:', 204903898720.0, 'Accuracy:', 89.702379703521729)
('Epoch', 5, 'completed out of', 100, 'loss:', 122495508064.0, 'Accuracy:', 90.035712718963623)
('Epoch', 6, 'completed out of', 100, 'loss:', 76852366736.0, 'Accuracy:', 90.321427583694458)
('Epoch', 7, 'completed out of', 100, 'loss:', 49968487256.0, 'Accuracy:', 90.77380895614624)
('Epoch', 8, 'completed out of', 100, 'loss:', 33380041192.0, 'Accuracy:', 90.976190567016602)
('Epoch', 9, 'completed out of', 100, 'loss:', 22769527384.0, 'Accuracy:', 90.559524297714233)
('Epoch', 10, 'completed out of', 100, 'loss:', 15787749192.0, 'Accuracy:', 90.416663885116577)
('Epoch', 11, 'completed out of', 100, 'loss

('Epoch', 85, 'completed out of', 100, 'loss:', 679504.73205566406, 'Accuracy:', 95.416665077209473)
('Epoch', 86, 'completed out of', 100, 'loss:', 670164.240234375, 'Accuracy:', 95.726191997528076)
('Epoch', 87, 'completed out of', 100, 'loss:', 664183.91003417969, 'Accuracy:', 95.261907577514648)
('Epoch', 88, 'completed out of', 100, 'loss:', 650240.58764648438, 'Accuracy:', 95.714282989501953)
('Epoch', 89, 'completed out of', 100, 'loss:', 641286.64453125, 'Accuracy:', 95.773810148239136)
('Epoch', 90, 'completed out of', 100, 'loss:', 635143.8896484375, 'Accuracy:', 95.369046926498413)
('Epoch', 91, 'completed out of', 100, 'loss:', 626909.142578125, 'Accuracy:', 96.119046211242676)
('Epoch', 92, 'completed out of', 100, 'loss:', 615854.22595214844, 'Accuracy:', 95.964282751083374)
('Epoch', 93, 'completed out of', 100, 'loss:', 610519.78405761719, 'Accuracy:', 95.345240831375122)
('Epoch', 94, 'completed out of', 100, 'loss:', 603387.1357421875, 'Accuracy:', 95.714282989501953)

In [31]:
# Accuracy_df= pd.DataFrame({"train accuracy(beta=0)": accuracy_train_b0,
#                            "test accuracy(beta=0.0)": accuracy_test_b0,
#                            "train accuracy(beta=0.01)": accuracy_train_b0_01,
#                            "test accuracy(beta=0.01)": accuracy_test_b0_01 }) 

In [33]:
# Accuracy_df.to_csv('accuracy(beta=0,0.01,epoch=100).csv', index=False)

In [ ]:
Accuracy_data = pd.read_csv('accuracy(beta=0,0.01,epoch=100).csv')
Accuracy_data.head()